In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# load image
def li(p):
    img = cv2.imread(p)
    img = img.astype(np.float32)/127.5 -1
    img = np.expand_dims(img,0)
    img = tf.convert_to_tensor(img)
    return img

# preprocess image
def pi(img,td=224):
    shp = tf.cast(tf.shape(img)[1:-1], tf.float32)
    sd  = min(shp)
    scl = td/sd
    nhp = tf.cast(shp*scl, tf.int32)
    img = tf.image.resize(img,nhp)
    img = tf.image.resize_with_crop_or_pad(img, td,td)
    return img

In [ ]:
m = '/kaggle/input/cartoongan/tflite/dr/1/1.tflite'

In [ ]:
stylized_output_folder = "/kaggle/working/cartoongan_op"
os.makedirs(stylized_output_folder, exist_ok=True)

def cartoon(img_p):
    # loading image
    si = li(img_p)

    psi = pi(si,td=512)
    psi.shape

    # model dataflow 
    
    i = tf.lite.Interpreter(model_path=m)
    ind = i.get_input_details()
    i.allocate_tensors()
    i.set_tensor(ind[0]['index'],psi)
    i.invoke()

    r = i.tensor(i.get_output_details()[0]['index'])()

    # post process the model output
    o = (np.squeeze(r)+1.0)*127.5
    o = np.clip(o,0,255).astype(np.uint8)
    o = cv2.cvtColor(o,cv2.COLOR_BGR2RGB)
    
    cv2.imwrite(os.path.join(stylized_output_folder, f"cartoon_frame_{frame_count}.jpg"), o)
    
#     print('input:')
#     plt.imshow(plt.imread(img_p))
#     plt.show();
#     print('output:')
#     plt.imshow(o)
#     plt.show();

In [ ]:
# INPUT VIDEO


video_path = "/kaggle/input/private-dataset/testvideo3.mp4"
output_folder = "/kaggle/working/temporary_video_frames"

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(total_frames)
user_defined_fps = 10

os.makedirs(output_folder, exist_ok=True)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter('output_video.mp4', fourcc, user_defined_fps, (int(cap.get(3)), int(cap.get(4))))

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    if frame_count % int(fps / user_defined_fps) == 0:

        cv2.imwrite(os.path.join(output_folder, f"frame_{frame_count}.jpg"), frame)        
        cartoon(f'{output_folder}/frame_{frame_count}.jpg')
        
        frame_image = cv2.imread(os.path.join(output_folder, f"frame_{frame_count}.jpg"))
        output_video.write(frame_image)


    # Print progress bar
    progress = frame_count / total_frames
    bar_length = 40
    progress_bar = int(bar_length * progress)
    percent = round(progress * 100, 2)
    bar = '=' * progress_bar + '.' * (bar_length - progress_bar)
    print(f'\rProgress: [{bar}] {percent}% Complete', end='', flush=True)

cap.release()
output_video.release()

# Print completion message
print("\nVideo processing completed.")

In [ ]:
stylized_images_dir = "/kaggle/working/cartoongan_op"
image_files = sorted(
    [os.path.join(stylized_images_dir, file) for file in os.listdir(stylized_images_dir)],
    key=lambda x: os.path.getmtime(x)
)

first_image = cv2.imread(image_files[0])
height, width, _ = first_image.shape

output_video_path = "/kaggle/working/output_video/output_stylized_video3.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter(output_video_path, fourcc, user_defined_fps, (width, height))

for image_file in image_files:
    image = cv2.imread(image_file)
    output_video.write(image)

output_video.release()

print(f"Video saved as: {output_video_path}")

In [ ]:
img = '/kaggle/working/cartoongan_op/cartoon_frame_92.jpg'

In [ ]:
print('input:')
plt.imshow(plt.imread(img))
plt.show();